In [1]:
import gym
import numpy as np
import random
from IPython.display import clear_output

In [2]:
env = gym.make('Marvin-v0')
action_space_size = env.action_space.shape[0]
state_space_size = env.observation_space.shape[0]

q_table = np.zeros((state_space_size, action_space_size))
print(action_space_size)
print(env.action_space)
print(env.action_space.high)
print(env.action_space.low)
print(state_space_size)
print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)
print(q_table)

4
Box(4,)
[1. 1. 1. 1.]
[-1. -1. -1. -1.]
24
Box(24,)
[inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf]
[-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [3]:
import random, math
random.seed(422142)

def mult_and_sum(a, b):
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    return result

def l_plus_c(a, c):
    result = []
    for i in range(len(a)):
        res = a[i] + c
        result.append(res)
    return result

class Neuron(object):
    def __init__(self, inputs_count):
        self.w = [random.random()] * inputs_count
        self.bias = random.random()
        
    def __str__(self):
        return [self.w, self.bias]
        
    def _net_input(self, data):
        return mult_and_sum(self.w, data) + self.bias
    
    def _activation_function(self, observation):
        net_input = self._net_input(observation)
        return 1 / (1 + math.exp(-(net_input)))
        
    # Returning 1 single number
    def predict(self, data):
        result = self._activation_function(data)
        return ((result - 0.5) * 2 if result >= 0.5 else (-result * 2))

    
    
    def accuracy(self, data, target):
        result = 0
        for X, y in zip(data, target):
            result += int(self.predict(X) == y)
        return result / len(target)    
    
    def learn(self, observation, reward, eta = 0.001):
        data = np.array(observation)
        e = eta * (reward - self.predict(data))
        self.w += data * e
        self.bias += e
        return

In [4]:
# create a network of three layers. Input -> 24 Neurons accept all features produces 1 single number each
# Second layer is consist of 24 Neurons as well -> we are sending results from first layer 
#                                                                to each neuron in second layer
# Third layer is consist of 4 Neurons. Each accepts all answers from Second layer and return 1 single number.
#                                                            Collection of thouse numbers is my action


In [5]:
class Layer(object):
    def __init__(self, input_counts):
        self.neurons = []
        self.previousLayer = None
        self.nextLayer = None
        
        for i in range(input_counts):
            self.neurons.append(Neuron(input_counts))
    
    def connectPrevious(self, previousLayer):
        self.previousLayer = previousLayer
    
    def connectTo(self, nextLayer):
        self.nextLayer = nextLayer
        nextLayer.connectPrevious(self)
    
    
    def run(self, observation):
        # Run through each Neuron and command them to make a prediction from given observation. 
        # return array of results
        result = []
        for neuron in self.neurons:
            result.append(neuron.predict(observation))
        return result
    
    
    def update(self, oldObservation, reward):
        # Run through each Neuron and command them to learn from data!
        for neuron in self.neurons:
            neuron.learn(oldObservation, reward)
    
    def howManyNeurons(self):
        return len(self.neurons)
        
    def printConnectedLayers(self):
        print(self.previousLayer)
        print(self.nextLayer)
    

In [6]:
l = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print(l)
print(l[0])
print(l[-1])
print(l[1:-1])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
9
[1, 2, 3, 4, 5, 6, 7, 8]


In [7]:
class Brain(object):
    def __init__(self, numberOfLayers, input_counts, output_counts):
        self.layers = []
        self.outputLayerW = [random.random()] * input_counts
        self.initializeLayers(numberOfLayers, input_counts, output_counts)
        
    def initializeLayers(self, quantity, neuronPerLayer, neuronPerOutput):
        for i in range (quantity):
            # Create quantity - 1 layers each with neuronPerLayer neurons 
            # Connect each layer
            freshLayer = Layer(neuronPerLayer) if i != quantity - 1 else Layer(neuronPerOutput)
            if (i != 0):    
                self.layers[-1].connectTo(freshLayer)
            self.layers.append(freshLayer)
    
    
    def learn(self, oldObservation, reward):
        # Run through each layer and command them to update!
        # I wish! Second layer should get data from first layer! 
        
        # Input layer
        input_layer = self.layers[0]
        
        # Hidden Layers 
        hidden_layers = self.layers[1: -1]
        
        # Output layer
        output_layer = self.layers[-1]
        
        # Comand the first layer to update himself
        input_layer.update(oldObservation, reward)
        
        # Update hidden Layers in a loop
        # run the layer to get new results
        results_for_hiddenLayer = input_layer.run(oldObservation)
        for hiddenLayer in hidden_layers:
                hiddenLayer.update(results_for_hiddenLayer, reward)
                results_for_hiddenLayer = hiddenLayer.run(results_for_hiddenLayer)

        # Get results from last hidden layer and use it to update the outputLayer
        
        results_for_outputLayer = results_for_hiddenLayer
        print("results for output! ", results_for_outputLayer)
        
        
    
    def chooseAction(self, observation):
        layer_output = []
        for layer in self.layers:
            # Run though each layer and command them to return array of all answers from all neurons in the layer
            layer_output.append(layer.run(observation))
    
    
    
    def howManyLayers(self):
        return len(self.layers)
    
    def getLayers(self):
        return self.layers

In [8]:
marvin = Brain(3, 24, 4)
print(marvin.howManyLayers())
layers = marvin.getLayers()
for layer in layers:
    print(layer.howManyNeurons())

3
24
24
4


In [9]:
# Next we integrate this nn in our enviroment. allow agent choose an action!

num_episodes = 10
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001




In [10]:
rewards_all_episodes = []



for episode in range(num_episodes):
    observation = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
         # Exploration-Explotation trade off
        env.render()
        exploration_rate_treshold = random.uniform(0,1)
        if exploration_rate_treshold >= exploration_rate:
            action = marvin.chooseAction(observation)
        else:
            action = env.action_space.sample()

        new_observation, reward, done, info = env.step(action)
        print(observation, reward, done, info)
        # Update my weights according to reward on each neuron in my brain.
        marvin.learn(observation, reward)
        rewards_current_episode += reward
        if done == True:
            break
        
    #Exploration decay logic 
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    rewards_all_episodes.append(rewards_current_episode) 


[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-02 -1.87642453e-03  9.34906870e-01  0.00000000e+00
  0.00000000e+00  5.72824292e-02 -1.87134231e-03  8.45454082e-01
 -7.76102146e-10  0.00000000e+00  6.56928718e-01  6.64389133e-01
  6.87641203e-01  7.29558408e-01  7.95953512e-01  8.97825241e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] -0.0314735995605588 False {}
results for output!  [0.9999997976110233, 0.9999999989551043, 0.9999999536599586, 0.9999999049006725, 0.9979866119214214, 0.9999998237748806, 0.9999998948444622, 0.642453924680858, 0.9999992883796178, 0.9982194637012503, 0.8509368693027006, 0.9998653027096274, 0.9999999802487125, 0.9999812388259821, 0.9998232447827298, 0.9999297613821545, 0.9999999912414745, 0.9999999423765285, 0.9999486156399362, 0.9964516787976312, 0.9999098521298251, 0.9996167534414744, 0.8872749952736496, 0.9999999970418303]
[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-

[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-02 -1.87642453e-03  9.34906870e-01  0.00000000e+00
  0.00000000e+00  5.72824292e-02 -1.87134231e-03  8.45454082e-01
 -7.76102146e-10  0.00000000e+00  6.56928718e-01  6.64389133e-01
  6.87641203e-01  7.29558408e-01  7.95953512e-01  8.97825241e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] -0.043804174207773 False {}
results for output!  [0.9999997220704921, 0.9999999985254715, 0.9999999360808871, 0.9999998692959804, 0.9973848702420491, 0.999999758751507, 0.9999998550438216, 0.5975956507793165, 0.9999990305879627, 0.9976905118031894, 0.8212287586239406, 0.999822200047725, 0.9999999725645119, 0.9999749444281503, 0.999767107859596, 0.9999071353403681, 0.9999999877715244, 0.9999999205853349, 0.999931735836205, 0.9954142246996567, 0.9998806560882934, 0.9994990135167987, 0.8628504093116207, 0.9999999958440475]
[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-02 -

[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-02 -1.87642453e-03  9.34906870e-01  0.00000000e+00
  0.00000000e+00  5.72824292e-02 -1.87134231e-03  8.45454082e-01
 -7.76102146e-10  0.00000000e+00  6.56928718e-01  6.64389133e-01
  6.87641203e-01  7.29558408e-01  7.95953512e-01  8.97825241e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] -0.156825319859512 False {}
results for output!  [0.9999995224834357, 0.9999999973408555, 0.9999998893097466, 0.9999997751000844, 0.9959323651208949, 0.9999995877917678, 0.9999997492529458, 0.5211145267287356, 0.9999983580430036, 0.996416368491952, 0.7624544429116737, 0.9997152678722925, 0.9999999518938159, 0.9999590325575314, 0.9996282507496166, 0.9998508543732707, 0.9999999783621394, 0.9999998626855624, 0.9998894227981996, 0.9929328563243072, 0.9998078804419404, 0.9992115158237744, 0.8127366246906564, 0.9999999925642742]
[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-0

[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-02 -1.87642453e-03  9.34906870e-01  0.00000000e+00
  0.00000000e+00  5.72824292e-02 -1.87134231e-03  8.45454082e-01
 -7.76102146e-10  0.00000000e+00  6.56928718e-01  6.64389133e-01
  6.87641203e-01  7.29558408e-01  7.95953512e-01  8.97825241e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] -0.009913321280229898 False {}
results for output!  [0.9999993503212168, 0.999999996278135, 0.9999998487053148, 0.9999996937487412, 0.994780937283624, 0.9999994409874491, 0.9999996574929417, 0.47950290990418054, 0.9999977846423775, 0.9954081554347964, 0.7251076148309223, 0.9996282443649087, 0.9999999337660639, 0.9999458553900462, 0.9995155573476393, 0.9998049380440568, 0.9999999700492643, 0.9999998124822189, 0.9998546544046738, 0.990983838607489, 0.9997483927240889, 0.9989809838343018, 0.7796191156103134, 0.9999999896406733]
[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e

[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-02 -1.87642453e-03  9.34906870e-01  0.00000000e+00
  0.00000000e+00  5.72824292e-02 -1.87134231e-03  8.45454082e-01
 -7.76102146e-10  0.00000000e+00  6.56928718e-01  6.64389133e-01
  6.87641203e-01  7.29558408e-01  7.95953512e-01  8.97825241e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] -0.009485327581565773 False {}
results for output!  [0.9999991179621808, 0.9999999947993028, 0.9999997936286182, 0.9999995838489091, 0.9933267960294008, 0.9999992435478642, 0.999999533114847, 0.43994887666659555, 0.9999970177503046, 0.9941365436973877, 0.6858124541463295, 0.9995159977312436, 0.999999908982651, 0.9999286261417017, 0.9993705142849565, 0.9997455972381353, 0.9999999586188109, 0.9999997444512359, 0.999809470162994, 0.9885411673178228, 0.9996713983891445, 0.99868706328283, 0.7437405216740827, 0.999999985592499]
[-1.42792333e-02 -4.67989920e-05  1.73622999e-03 -1.59917605e-02
  8.70858878e-02

...

[-1.42588541e-02  5.55035379e-05 -4.70535815e-03 -1.59972680e-02
  8.78763199e-02  1.00421123e-02  9.34906840e-01 -7.19309909e-03
  0.00000000e+00  5.86512722e-02  1.01520661e-02  8.43884274e-01
 -7.42286071e-03  0.00000000e+00  6.56923413e-01  6.64383769e-01
  6.87635660e-01  7.29552567e-01  7.95947134e-01  8.97818029e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] -0.18142666888237002 False {}
results for output!  [0.3371163375832198, 0.7273016197723723, 0.5596538993339795, 0.5144762346980991, -0.7530457462673412, 0.4919334410608349, 0.41207055568587214, 0.03691267984200164, 0.23019917749981733, -0.9455927733362289, 0.023272451982962128, -0.9014828805148811, 0.5618436945294509, -0.9664358837567703, -0.9092930269282302, -0.9879990557198656, 0.5989722611453447, 0.5387240068536594, -0.8667934050672917, -0.8370082016792639, -0.8817533053419133, -0.9999735054264457, 0.009115620988294104, 0.6547773922933386]
[-1.42588541e-02  5.55035379e-05 -4.70535815e-03 -1.59972680

[-1.42588541e-02  5.55035379e-05 -4.70535815e-03 -1.59972680e-02
  8.78763199e-02  1.00421123e-02  9.34906840e-01 -7.19309909e-03
  0.00000000e+00  5.86512722e-02  1.01520661e-02  8.43884274e-01
 -7.42286071e-03  0.00000000e+00  6.56923413e-01  6.64383769e-01
  6.87635660e-01  7.29552567e-01  7.95947134e-01  8.97818029e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] -0.20749445296327393 False {}
results for output!  [-0.044524803716523356, -0.01891254920946135, -0.052746559737048476, -0.06064551106750804, -0.24404432176343172, -0.07492118151258283, -0.041813173983981045, 0.005592706696909255, -0.05778510580060918, -0.3855649200483461, -0.9283479713763817, -0.16672284421289538, -0.034214948181117005, -0.10509128282638681, -0.18521645486488697, -0.1929348688415935, -0.02696312708923973, -0.053244379589054154, -0.11174503996990792, -0.3467008931185125, -0.13744024302996874, -0.3368208229793742, -0.8330421807668067, -0.020761801947989277]
[-1.42588541e-02  5.55035379e

[-1.42588541e-02  5.55035379e-05 -4.70535815e-03 -1.59972680e-02
  8.78763199e-02  1.00421123e-02  9.34906840e-01 -7.19309909e-03
  0.00000000e+00  5.86512722e-02  1.01520661e-02  8.43884274e-01
 -7.42286071e-03  0.00000000e+00  6.56923413e-01  6.64383769e-01
  6.87635660e-01  7.29552567e-01  7.95947134e-01  8.97818029e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] 0.0037969520886727177 False {}
results for output!  [0.49608667528623496, 0.8301517398118021, 0.6832259797047076, 0.6391861678737591, -0.8603094895419029, 0.6149794916218239, 0.5582184467109652, 0.0588241418343165, 0.3961842761641199, 0.03742053387647504, 0.06658223757267723, 0.023068905932914552, 0.6917075229886207, 0.11454988070937877, 0.027242890873417025, 0.1161224367244893, 0.7258107819071797, 0.6647439494122866, 0.002378149325140244, -0.9421524534619987, 0.008976265232188307, 0.10384497431338002, 0.06660574646676021, 0.7740396192617769]
[-1.42588541e-02  5.55035379e-05 -4.70535815e-03 -1.59972680

[-1.42588541e-02  5.55035379e-05 -4.70535815e-03 -1.59972680e-02
  8.78763199e-02  1.00421123e-02  9.34906840e-01 -7.19309909e-03
  0.00000000e+00  5.86512722e-02  1.01520661e-02  8.43884274e-01
 -7.42286071e-03  0.00000000e+00  6.56923413e-01  6.64383769e-01
  6.87635660e-01  7.29552567e-01  7.95947134e-01  8.97818029e-01
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00] 0.0660730591416359 False {}
results for output!  [0.47279144122923644, 0.8133037301612458, 0.6632585182665913, 0.6184020505327377, -0.9122274023812933, 0.5947785583301415, 0.5332384187104064, 0.06748540352827925, 0.3781860389844407, 0.04763546294962717, 0.07421418156516202, 0.031644399532500644, 0.669396335838923, 0.11742862542763022, 0.03585952732098874, 0.12342174810920215, 0.7036232735895318, 0.6442367758884615, 0.011278076801247261, -0.9986167723437975, 0.01749292762245247, 0.11399279147092489, 0.07154864117567028, 0.7533696341561387]
[-1.42588541e-02  5.55035379e-05 -4.70535815e-03 -1.59972680e-02

TypeError: 'NoneType' object is not subscriptable

In [43]:
import random, math
random.seed(422142)

def mult_and_sum(a, b):
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    return result

def l_plus_c(a, c):
    result = []
    for i in range(len(a)):
        res = a[i] + c
        result.append(res)
    return result

class Marvin(object):
    def __init__(self, inputs_count):
        self.w = [random.random()] * inputs_count
        self.bias = random.random()
        
    def _net_input(self, data):
        return mult_and_sum(self.w, data) + self.bias
    
    def _activation_function(self, observation):
        net_input = self._net_input(observation)
        return 1 / (1 + math.exp(-(net_input)))
        
    # Returning 1 single number
    def predict(self, data):
        result = self._activation_function(data)
        return ((result - 0.5) * 2 if result >= 0.5 else (-result * 2))

    def accuracy(self, data, target):
        result = 0
        for X, y in zip(data, target):
            result += int(self.predict(X) == y)
        return result / len(target)    
    
    def learn(self, observation, action, reward, eta = 0.001):
        data = np.concatenate((observation, action), axis=None)
        e = eta * (reward - self.predict(data))
        self.w += data * e
        self.bias += e
        return

In [44]:
observation = env.reset()
action = env.action_space.sample()
print(type(observation), type(action))
print(action_space_size + state_space_size)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
28


In [58]:
robot = Marvin(action_space_size + state_space_size)

for j in range(50000):
    observation = env.reset()
    for i in range(20):
        action = env.action_space.sample()
        observation_new, reward, done, info = env.step(action)
    #     print(reward)
        if (reward <= 1 and reward >= -1):
            robot.learn(observation, action, reward, 0.0001)
        else:
            break
        observation = observation_new

observation = env.reset()
X = []
y = []
for k in range(10):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    data = np.concatenate((observation, action), axis=None)
    X.append(data)
    y.append(reward)
    
#print(X)
#print(y)
print(robot.predict(X[0]), y[0])
print(robot.accuracy(X, y))


0.009307011625510508 -0.07481399103750666
0.0


In [59]:
for k in range(10):
    print(robot.predict(X[k]), y[k])

0.009307011625510508 -0.07481399103750666
0.007328073585658901 -0.11518605728447437
0.004950032265731208 -0.08263521004716674
0.005069243288558178 -0.08054902566472807
0.0027335650575319725 -0.1265358516275883
0.0014251222912342598 -0.20129821601510048
0.004863242205111584 0.01990209116538604
0.00527117037000302 0.07903253786265851
-0.9999082724839304 0.08354698637375993
0.006463804152735841 0.045246084546051814


In [ ]:
observation = env.reset()
for limb in robot:
    print(limb.predict(observation))

In [ ]:
state = env.reset()
print(state)


In [ ]:
num_episodes = 10
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [ ]:
rewards_all_episodes = []

#Q-learning algorithm
for episode in range(num_episodes):
    state = env.reset()
    
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        env.render()
        # Exploration-Explotation trade off
#         exploration_rate_treshold = random.uniform(0,1)
#         if exploration_rate_treshold >= exploration_rate:
#             action = np.argmax(q_table[state])
#         else:
#             action = env.action_space.sample()
        
#         print("action = ", action)
        action = []
        for limb in robot:
            action.append(limb.predict(observation))
        observation, reward, done, info = env.step(action)
#         print(observation, reward, done, info)
        # Update Q-table
#         q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
#             learning_rate * (reward  + discount_rate * np.max(q_table[state]))
#         state = observation
#         rewards_current_episode += reward
        if done == True or reward == -100:
            break
            
#     #Exploration decay logic 
#     exploration_rate = min_exploration_rate + \
#         (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
#     rewards_all_episodes.append(rewards_current_episode)

# #Calculate and print the average reward per 1000 episodes
# rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes / 1000)
# count = 1000
# print("*** Average reward per 1000 episodes ***\n")
# for r in rewards_per_thousand_episodes:
#     print(count, ": ", str(sum(r/ 1000)))
#     count += 1000
    

# #Print updated Q-table
# print("\n\n*****Q-table*****\n\n")
# print(q_table)

In [ ]:
import random, math
random.seed(422142)

def mult_and_sum(a, b):
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    return result

class Marvin(object):
    def __init__(self, inputs_count):
        self.w = [random.random()] * inputs_count
        self.bias = random.random()
        
    def _net_input(self, observation):
        return mult_and_sum(self.w, observation) + self.bias
    
    def _activation_function(self, observation):
        net_input = self._net_input(observation)
        return 1 / (1 + math.exp(-(net_input)))
        
    def predict(self, data):
        result = self._activation_function(data)
        return ((result - 0.5) * 2 if result >= 0.5 else (-result * 2))

    def learn(self, observation, action, reward, eta = 0.001):
        e = reward - (self.predict(observation) - action)
        self.w = self.w + eta * e * observation
        self.bias = self.bias + eta * e
        return


    robot = [Marvin(24), Marvin(24), Marvin(24), Marvin(24)]

for j in range(7000):
    observation = env.reset()
    for i in range(100):
        action = env.action_space.sample()
        observation_new, reward, done, info = env.step(action)
    #     print(reward)
        if (reward <= 1 and reward >= -1):
            for index in range(len(robot)):
                robot[index].learn(observation, action[index], reward)
        else:
            break
        observation = observation_new
    
